In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()
spark


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,353 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,998 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [2]:
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",",header=True, inferSchema = True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
df.printSchema()
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql import functions as F
df = df.withColumn("price", df["price"].cast(IntegerType()))
df = df.withColumn("bedrooms", df["bedrooms"].cast(IntegerType()))
df = df.withColumn("date", F.to_date(df["date"]))
df.select(F.max('date')).show()
df.select(F.min('date')).show()
date1 =('2019-01-01', '2019-12-31')
date2 = ('2020-01-01', '2020-12-30')
date3=('2021-01-01', '2021-01-30')
date4=('2022-01-01', '2022-06-14')

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)

+----------+
| max(date)|
+----------+
|2022-06-14|
+----------+

+----------+
| min(date)|
+----------+
|2019-01-01|
+----------+



In [7]:
from pyspark.sql import functions as F
from pyspark.sql.functions import round
from pyspark.sql.functions import year
df.select(year(df["date"]))
df=df.withColumn("year", year(df['date']))
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
year_price_4bedroom =df.select(["price", "date","bedrooms"]).where(df.bedrooms == 4)
new2019df=df.groupBy(df.date.between('2019-01-01', '2019-12-31').alias("Year=2019")).agg(round(F.avg(F.when(df.bedrooms == F.lit('4'), F.col("price"))),2).alias("average_price")).show(truncate=False)
new2020df=df.groupBy(df.year == F.lit('2020')).agg(round(F.avg(F.when(df.bedrooms == F.lit('4'), F.col("price"))), 2).alias("average_price")).show()
new2021df=df.groupBy(df.year == F.lit('2021')).agg(round(F.avg(F.when(df.bedrooms == F.lit('4'), F.col("price"))), 2).alias("average_price")).show()
new2022df=df.groupBy(df.year == F.lit('2022')).agg(round(F.avg(F.when(df.bedrooms == F.lit('4'), F.col("price"))), 2).alias("average_price")).show()
new2019df
new2020df
new2021df
new2022df

+---------+-------------+
|Year=2019|average_price|
+---------+-------------+
|true     |300263.7     |
|false    |299417.37    |
+---------+-------------+

+-------------+-------------+
|(year = 2020)|average_price|
+-------------+-------------+
|         true|    298353.78|
|        false|    300197.81|
+-------------+-------------+

+-------------+-------------+
|(year = 2021)|average_price|
+-------------+-------------+
|         true|    301819.44|
|        false|    298769.09|
+-------------+-------------+

+-------------+-------------+
|(year = 2022)|average_price|
+-------------+-------------+
|         true|    296363.88|
|        false|    300147.68|
+-------------+-------------+



In [8]:
_# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
bath_bedrooms_avgprice = df.groupBy(df.date_built, F.when(df.bedrooms== 3, 1).when(df.bathrooms==3, 1).otherwise(0).alias("3Bedrooms&bathrooms")).agg(round(F.avg(df.price),2).alias("average_price"))
bath_bedrooms_avgprice.show()

+----------+-------------------+-------------+
|date_built|3Bedrooms&bathrooms|average_price|
+----------+-------------------+-------------+
|      2012|                  0|    338716.63|
|      2011|                  0|     339922.0|
|      2014|                  0|    341867.73|
|      2017|                  0|    342863.35|
|      2013|                  0|    338539.26|
|      2010|                  0|    330051.36|
|      2014|                  1|    302781.87|
|      2015|                  0|     339105.1|
|      2016|                  0|    338365.96|
|      2016|                  1|    303311.05|
|      2012|                  1|    300270.37|
|      2013|                  1|    306642.66|
|      2015|                  1|    302997.92|
|      2011|                  1|    302751.77|
|      2010|                  1|    299133.19|
|      2017|                  1|    301733.01|
+----------+-------------------+-------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
home_sales_df = df.groupBy(df.date_built, F.when(df.bedrooms== 3, 1).when(df.bathrooms==3, 1).otherwise(0).alias("3Bedrooms&bathrooms"),
                                    F.when(df.floors == 2, 1).when(df.sqft_living >= 2000, 1).otherwise(0).alias("2floors_morethan2000SQFT")).agg(round(F.avg(df.price),2).alias("Average_Price"))
home_sales_df.show()

+----------+-------------------+------------------------+-------------+
|date_built|3Bedrooms&bathrooms|2floors_morethan2000SQFT|Average_Price|
+----------+-------------------+------------------------+-------------+
|      2013|                  1|                       0|     285754.7|
|      2014|                  1|                       1|    306821.56|
|      2010|                  0|                       1|    343016.82|
|      2012|                  0|                       1|    354737.58|
|      2015|                  1|                       0|    281701.47|
|      2016|                  0|                       1|     355089.8|
|      2011|                  0|                       0|    291042.71|
|      2013|                  0|                       1|    355952.04|
|      2016|                  1|                       1|    309764.75|
|      2011|                  1|                       1|    308540.15|
|      2012|                  1|                       0|    280

In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()
home_sales_HS = df.groupBy(df.view, F.when(df.bedrooms== 3, 1).when(df.bathrooms==3, 1).otherwise(0).alias("3Bedrooms&bathrooms"),
                                    F.when(df.floors == 2, 1).when(df.sqft_living >= 2000, 1).otherwise(0).alias("2floors_morethan2000SQFT")).agg(round(F.avg(df.price),2).alias("Average_Price"))
home_sales_HS.orderBy(F.when(home_sales_HS.Average_Price >= 350000, 1).otherwise(0).desc()).show()
print("--- %s seconds ---" % (time.time() - start_time))
Schema_homesales = home_sales_HS.schema
Schema_homesales


+----+-------------------+------------------------+-------------+
|view|3Bedrooms&bathrooms|2floors_morethan2000SQFT|Average_Price|
+----+-------------------+------------------------+-------------+
|  83|                  1|                       1|    967777.94|
|  60|                  1|                       1|    799710.71|
|  76|                  1|                       1|   1013447.44|
|  70|                  0|                       1|    715162.21|
|  88|                  1|                       1|    916647.93|
|  75|                  0|                       1|    1155976.5|
|  89|                  1|                       1|     976732.5|
|  69|                  1|                       1|    699738.22|
|  92|                  1|                       1|    983242.27|
|  51|                  1|                       1|    807649.14|
|  58|                  1|                       1|    782675.21|
|  98|                  0|                       1|   1163281.95|
|  98|    

StructType([StructField('view', IntegerType(), True), StructField('3Bedrooms&bathrooms', IntegerType(), False), StructField('2floors_morethan2000SQFT', IntegerType(), False), StructField('Average_Price', DoubleType(), True)])

In [11]:
# 7. Cache the temporary table home_sales.
df.createOrReplaceTempView("home_sales")
spark.sql("""SELECT * FROM home_sales""").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

False

In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df_homesales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header=True)
df_homesales.write.parquet("/filestore/data/hsp", mode = "overwrite", partitionBy='date_built')
# this command above to write a parquet file created a folder called hsp  with datebuilt partion file stored in it.
# not sure how to add the folder to github. it is not downloadable. copy path shows /content/hsp

In [19]:
# 11. Read the formatted parquet data
homesales_new = spark.read.parquet('/filestore/data/hsp')


In [20]:
# 12. Create a temporary table for the parquet data.
homesales_new.createOrReplaceTempView('date_built_new')

In [21]:
start_time = time.time()
spark.sql("""select * from date_built_new """).show()
print("----%s seconds -----" % (time.time() - start_time))


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|